# Тонкая настройка моделей Open AI

Этот блокнот основан на актуальных рекомендациях из документации [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) от Open AI.

Тонкая настройка улучшает работу базовых моделей для вашего приложения за счёт дополнительного обучения на данных и контексте, которые важны для конкретного случая использования. Обратите внимание, что такие техники, как _few shot learning_ и _retrieval augmented generation_, позволяют обогатить стандартный запрос релевантными данными для повышения качества. Однако эти подходы ограничены максимальным размером окна токенов выбранной базовой модели.

При тонкой настройке мы фактически дообучаем саму модель на нужных данных (что позволяет использовать гораздо больше примеров, чем помещается в максимальное окно токенов) и развёртываем _кастомную_ версию модели, которой больше не нужно предоставлять примеры во время инференса. Это не только повышает эффективность проектирования запросов (у нас появляется больше свободы в использовании окна токенов для других целей), но и потенциально снижает затраты (за счёт уменьшения количества токенов, которые нужно отправлять модели при инференсе).

Тонкая настройка включает 4 шага:
1. Подготовить обучающие данные и загрузить их.
1. Запустить обучение для получения дообученной модели.
1. Оценить дообученную модель и доработать её для повышения качества.
1. Развернуть дообученную модель для инференса, когда результат вас устроит.

Обратите внимание, что не все базовые модели поддерживают тонкую настройку — [проверьте документацию OpenAI](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) для получения самой свежей информации. Также можно дообучать уже дообученную модель. В этом уроке мы будем использовать `gpt-35-turbo` в качестве целевой базовой модели для тонкой настройки.

---


### Шаг 1.1: Подготовьте ваш датасет

Давайте создадим чат-бота, который поможет вам разобраться в периодической таблице элементов, отвечая на вопросы о каждом элементе в форме лимерика. В _этом_ простом уроке мы просто создадим датасет для обучения модели с несколькими примерами ответов, чтобы показать ожидаемый формат данных. В реальном проекте вам потребуется собрать датасет с гораздо большим количеством примеров. Также вы можете использовать открытый датасет (по вашей предметной области), если такой существует, и отформатировать его для использования в дообучении.

Поскольку мы работаем с `gpt-35-turbo` и ожидаем однократный ответ (chat completion), мы можем создать примеры, используя [этот рекомендуемый формат](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst), соответствующий требованиям OpenAI для chat completion. Если вы ожидаете диалог с несколькими репликами, используйте [формат для многотуровых примеров](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst), который включает параметр `weight`, чтобы указать, какие сообщения использовать (или не использовать) в процессе дообучения.

В этом уроке мы воспользуемся более простым однократным форматом. Данные представлены в [формате jsonl](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) — по одной записи на строку, каждая из которых оформлена как объект в формате JSON. В примере ниже показаны 2 записи — полный набор примеров (10 штук), который мы будем использовать для нашего урока по дообучению, смотрите в [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl). **Обратите внимание:** Каждая запись _должна_ быть определена в одной строке (не разбивайте на несколько строк, как это обычно делается в отформатированном JSON-файле)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

В реальном проекте для хороших результатов потребуется гораздо больше примеров — здесь нужно найти баланс между качеством ответов и временем/стоимостью дообучения. Мы используем небольшой набор, чтобы быстро пройти процесс дообучения и показать, как это работает. Более сложный пример дообучения смотрите в [этом примере из OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst).


### Шаг 1.2 Загрузка вашего набора данных

Загрузите данные с помощью Files API [как описано здесь](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Обратите внимание, что для запуска этого кода необходимо сначала выполнить следующие шаги:
 - Установить Python-пакет `openai` (убедитесь, что используете версию >=0.28.0 для поддержки последних функций)
 - Установить переменную окружения `OPENAI_API_KEY` со своим ключом OpenAI API
Чтобы узнать больше, ознакомьтесь с [руководством по настройке](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst), предоставленным для курса.

Теперь запустите код для создания файла для загрузки из вашего локального файла JSONL.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


### Шаг 2.1: Создайте задачу дообучения с помощью SDK


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


### Шаг 2.2: Проверьте статус задания

Вот что можно сделать с API `client.fine_tuning.jobs`:
- `client.fine_tuning.jobs.list(limit=<n>)` — Показать последние n заданий по дообучению
- `client.fine_tuning.jobs.retrieve(<job_id>)` — Получить подробную информацию о конкретном задании по дообучению
- `client.fine_tuning.jobs.cancel(<job_id>)` — Отменить задание по дообучению
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` — Показать до n событий из задания
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

Первый шаг этого процесса — _проверка обучающего файла_, чтобы убедиться, что данные имеют правильный формат.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


### Шаг 2.3: Отслеживайте события для мониторинга прогресса


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Шаг 2.4: Просмотрите статус в панели управления OpenAI


Вы также можете просмотреть статус, посетив сайт OpenAI и перейдя в раздел _Fine-tuning_ на платформе. Здесь отображается статус текущей задачи, а также можно отследить историю предыдущих запусков. На этом скриншоте видно, что предыдущий запуск завершился с ошибкой, а второй прошёл успешно. Для справки: это произошло потому, что в первом запуске использовался JSON-файл с некорректно отформатированными записями — после исправления второй запуск завершился успешно, и модель стала доступна для использования.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.ru.png)


Вы также можете просматривать сообщения о состоянии и метрики, прокручивая ниже на визуальной панели, как показано:

| Сообщения | Метрики |
|:---|:---|
| ![Сообщения](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.ru.png) |  ![Метрики](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.ru.png)|


### Шаг 3.1: Получение ID и тестирование дообученной модели в коде


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


### Шаг 3.2: Загрузка и тестирование дообученной модели в Playground

Теперь вы можете протестировать дообученную модель двумя способами. Во-первых, вы можете зайти в Playground и выбрать вашу новую дообученную модель из выпадающего списка моделей. Второй способ — воспользоваться опцией "Playground" в панели Fine-tuning (см. скриншот выше), которая открывает _сравнительный_ режим: версии базовой и дообученной модели отображаются рядом для быстрой оценки.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016497d39ced3d84ea296eec89073503f2bf346ec9718f913b5.ru.png)

Просто заполните системный контекст, который использовался в ваших обучающих данных, и введите тестовый вопрос. Вы заметите, что обе стороны обновляются с одинаковым контекстом и вопросом. Запустите сравнение, и вы увидите разницу в ответах между ними. _Обратите внимание, что дообученная модель формирует ответ в том формате, который вы задали в примерах, а базовая модель просто следует системному промпту_.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350c227e05700a47a89002d132949a56fa4ff37f266ebe997b2.ru.png)

Также вы увидите, что сравнение показывает количество токенов для каждой модели и время, затраченное на инференс. **Этот конкретный пример очень простой и предназначен для демонстрации процесса, но не отражает реальный датасет или сценарий**. Вы можете заметить, что в обоих случаях количество токенов одинаковое (системный контекст и пользовательский запрос идентичны), но дообученная модель тратит больше времени на инференс (кастомная модель).

В реальных сценариях вы не будете использовать такой учебный пример, а будете дообучать модель на реальных данных (например, каталог товаров для поддержки клиентов), где качество ответа будет заметно выше. В _таком_ случае, чтобы добиться аналогичного качества ответа от базовой модели, потребуется более сложная настройка промпта, что увеличит расход токенов и, возможно, время обработки инференса. _Чтобы попробовать это на практике, посмотрите примеры дообучения в OpenAI Cookbook._



---

**Отказ от ответственности**:  
Этот документ был переведен с помощью сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Несмотря на наши усилия обеспечить точность, автоматические переводы могут содержать ошибки или неточности. Оригинальный документ на исходном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется профессиональный перевод человеком. Мы не несем ответственности за любые недоразумения или неправильные толкования, возникшие в результате использования данного перевода.
